## Import data from csv

In [66]:
import pandas as pd
import numpy as np
import pickle

In [67]:
df0 = pd.read_csv("../grab-ai-safety-data/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df1 = pd.read_csv("../grab-ai-safety-data/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df2 = pd.read_csv("../grab-ai-safety-data/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df3 = pd.read_csv("../grab-ai-safety-data/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df4 = pd.read_csv("../grab-ai-safety-data/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df5 = pd.read_csv("../grab-ai-safety-data/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df6 = pd.read_csv("../grab-ai-safety-data/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df7 = pd.read_csv("../grab-ai-safety-data/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df8 = pd.read_csv("../grab-ai-safety-data/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df9 = pd.read_csv("../grab-ai-safety-data/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
response = pd.read_csv("../grab-ai-safety-data/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [68]:
df_features = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0)

In [69]:
df = pd.merge(
    df_features,
    response,
    how="left",
    on="bookingID"
)

## Feature Extraction

In [70]:
def weighted_sum(x, y):
    return (x * y).sum()

In [71]:
df["max_acceleration"] = df[["acceleration_x", "acceleration_y", "acceleration_z"]].max(axis=1)
df["max_gyro"] = df[["gyro_x", "gyro_y", "gyro_z"]].max(axis=1)
df["Speed_abs"] = np.abs(df["Speed"])

In [72]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label,max_acceleration,max_gyro,Speed_abs
0,1194000908341,4.000,203.000000,1.001973,9.745000,-0.790087,-0.066046,0.019175,0.044741,246.0,6.480000,1,9.745000,0.044741,6.480000
1,1099511627891,3.000,311.000000,0.452518,9.273026,3.098191,0.005382,0.002155,-0.001436,66.0,0.340365,0,9.273026,0.005382,0.340365
2,962072674446,10.000,142.807999,0.018542,-8.041156,-4.960114,0.004223,-0.014875,0.007397,1552.0,2.050000,0,0.018542,0.007397,2.050000
3,1142461300867,8.000,171.205292,-0.290698,-8.295367,-3.049042,0.033618,0.046224,0.030559,277.0,17.608448,0,-0.290698,0.046224,17.608448
4,412316860548,19.379,0.000000,0.675613,8.846130,1.852508,-0.008652,0.003693,0.001160,148.0,0.000000,0,8.846130,0.003693,0.000000


In [73]:
df_sorted = df.sort_values(["bookingID", "second"])
df_sorted["time_diffs"] = df_sorted["second"].diff()
df_sorted["Bearing_diffs"] = df_sorted["Bearing"].diff()
mask = df_sorted["bookingID"] != df_sorted["bookingID"].shift(1)
df_sorted["time_diffs"][mask] = 0
df_sorted["Bearing_diffs"][mask] = 0
df_sorted["distance_covered"] = df_sorted["Speed_abs"] * df_sorted["time_diffs"]
df_sorted["Bearing_diffs"] = np.abs(df_sorted["Bearing_diffs"])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [74]:
df_sorted.head(100)

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label,max_acceleration,max_gyro,Speed_abs,time_diffs,Bearing_diffs,distance_covered
9232561,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991,0,0.818112,0.070732,3.442991,0.0,0.000000,0.000000
10406491,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454,0,0.546405,0.043187,0.228454,1.0,0.000000,0.228454
1783254,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,0,-1.209448,0.015390,0.228454,1.0,0.000000,0.228454
7886694,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454,0,-0.598145,0.021902,0.228454,2.0,0.000000,0.456909
13281304,0,8.0,143.298294,-0.608313,-9.539658,-1.794583,-0.007538,-0.023838,0.018068,5.0,0.228454,0,-0.608313,0.018068,0.228454,1.0,0.000000,0.228454
11107740,0,8.0,143.298294,-0.867758,-9.698615,-1.615439,0.022728,-0.012178,0.005982,6.0,0.228454,0,-0.867758,0.022728,0.228454,1.0,0.000000,0.228454
8845149,0,8.0,143.298294,-1.050790,-9.745270,-1.411771,0.027603,0.001841,0.000904,7.0,0.228454,0,-1.050790,0.027603,0.228454,1.0,0.000000,0.228454
4544788,0,8.0,143.298294,-0.721213,-9.960004,-1.202271,0.001864,-0.007702,0.014018,8.0,0.228454,0,-0.721213,0.014018,0.228454,1.0,0.000000,0.228454
4177606,0,8.0,143.298294,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,9.0,0.228454,0,-0.346924,0.025118,0.228454,1.0,0.000000,0.228454
12543133,0,8.0,143.298294,0.294586,-10.085315,-1.531100,0.004587,-0.077703,0.069096,10.0,0.228454,0,0.294586,0.069096,0.228454,1.0,0.000000,0.228454


In [75]:
# Speed related features
df_sorted["std_speed"] = df_sorted["Speed"]
df_sorted["max_speed"] = df_sorted["Speed"]
df_sorted["min_speed"] = df_sorted["Speed"]
df_sorted["speed_10"] = df_sorted["Speed"]
df_sorted["speed_30"] = df_sorted["Speed"]
df_sorted["speed_70"] = df_sorted["Speed"]
df_sorted["speed_90"] = df_sorted["Speed"]

# Acceleration related features
df_sorted["avg_acceleration"] = df_sorted["max_acceleration"]
df_sorted["min_acceleration"] = df_sorted["max_acceleration"]
df_sorted["acc_10"] = df_sorted["max_acceleration"]
df_sorted["acc_30"] = df_sorted["max_acceleration"]
df_sorted["acc_70"] = df_sorted["max_acceleration"]
df_sorted["acc_90"] = df_sorted["max_acceleration"]
df_sorted["std_acc"] = df_sorted["max_acceleration"]

# Gyro related features
df_sorted["std_gyro"] = df_sorted["max_gyro"]
df_sorted["avg_gyro"] = df_sorted["max_gyro"]
df_sorted["min_gyro"] = df_sorted["max_gyro"]
df_sorted["std_gyro"] = df_sorted["max_gyro"]
df_sorted["gyro_10"] = df_sorted["max_gyro"]
df_sorted["gyro_30"] = df_sorted["max_gyro"]
df_sorted["gyro_70"] = df_sorted["max_gyro"]
df_sorted["gyro_90"] = df_sorted["max_gyro"]

# Bearing related features
df_sorted["std_bearing"] = df_sorted["Bearing_diffs"]
df_sorted["mean_bearing"] = df_sorted["Bearing_diffs"]
df_sorted["max_bearing"] = df_sorted["Bearing_diffs"]
df_sorted["min_bearing"] = df_sorted["Bearing_diffs"]
df_sorted["bearing_10"] = df_sorted["Bearing_diffs"]
df_sorted["bearing_30"] = df_sorted["Bearing_diffs"]
df_sorted["bearing_70"] = df_sorted["Bearing_diffs"]
df_sorted["bearing_90"] = df_sorted["Bearing_diffs"]

In [76]:
df_sorted.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,...,gyro_70,gyro_90,std_bearing,mean_bearing,max_bearing,min_bearing,bearing_10,bearing_30,bearing_70,bearing_90
9232561,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,...,0.070732,0.070732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10406491,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,...,0.043187,0.043187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1783254,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,...,0.015390,0.015390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7886694,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,...,0.021902,0.021902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13281304,0,8.0,143.298294,-0.608313,-9.539658,-1.794583,-0.007538,-0.023838,0.018068,5.0,...,0.018068,0.018068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
df_grouped = df_sorted.groupby("bookingID").agg({
    "second": "max",
    "distance_covered": "sum",
    "Speed": "mean",
    "std_speed": "std",
    "max_speed": "max",
    "min_speed": "min",
    "speed_10": lambda x: np.percentile(x, q=10),
    "speed_30": lambda x: np.percentile(x, q=30),
    "speed_70": lambda x: np.percentile(x, q=70),
    "speed_90": lambda x: np.percentile(x, q=90),
    "avg_acceleration": "mean",
    "min_acceleration": "min",
    "max_acceleration": "max",
    "std_acc": "std",
    "acc_10": lambda x: np.percentile(x, q=10),
    "acc_30": lambda x: np.percentile(x, q=30),
    "acc_70": lambda x: np.percentile(x, q=70),
    "acc_90": lambda x: np.percentile(x, q=90),
    "avg_gyro": "mean",
    "min_gyro": "min",
    "max_gyro": "max",
    "std_gyro": "std",
    "gyro_10": lambda x: np.percentile(x, q=10),
    "gyro_30": lambda x: np.percentile(x, q=30),
    "gyro_70": lambda x: np.percentile(x, q=70),
    "gyro_90": lambda x: np.percentile(x, q=90), 
    "Bearing_diffs": "mean",
    "min_bearing": "min",
    "max_bearing": "max",
    "std_bearing": "std",
    "bearing_10": lambda x: np.percentile(x, q=10),
    "bearing_30": lambda x: np.percentile(x, q=30),
    "bearing_70": lambda x: np.percentile(x, q=70),
    "bearing_90": lambda x: np.percentile(x, q=90), 
    "label": "max"
})

In [78]:
df_grouped.head()

,second,distance_covered,Speed,std_speed,max_speed,min_speed,avg_acceleration,max_acceleration,std_acc,avg_gyro,max_gyro,std_gyro,Bearing_diffs,max_bearing,std_bearing,label
bookingID,,,,,,,,,,,,,,,,
0,1589.0,9576.396033,8.901840,7.215633,22.946083,-1.0,-0.455843,4.782614,0.817974,0.048507,0.469724,0.065931,4.129184,358.976575,31.363908,0
1,1034.0,6942.828210,7.904379,7.089165,21.882141,-1.0,9.544505,12.536156,0.517977,0.025951,0.470837,0.051046,6.906040,311.000000,31.295836,1
2,825.0,616.110921,3.159100,2.930354,9.360483,0.0,9.855846,13.333716,0.506595,0.045989,0.431695,0.073266,8.888889,353.000000,34.648900,1
4,1094.0,6715.840005,6.201331,5.633055,19.780001,0.0,-0.355672,1.805051,0.492215,0.062416,0.652628,0.077818,4.119712,348.695439,26.918189,1
6,1094.0,5056.750774,4.622729,5.319819,16.293852,0.0,9.534246,13.249917,0.620064,0.052584,0.532123,0.072554,4.569556,353.000000,27.122535,0


In [105]:
with open('outputs/df_cleaned.pickle', 'wb') as f:
    pickle.dump(df_grouped, f)